In [ ]:
from langchain.utilities import WikipediaAPIWrapper, DuckDuckGoSearchAPIWrapper
from langchain.tools import WikipediaQueryRun, DuckDuckGoSearchResults
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import SystemMessage
from langchain.document_loaders import WebBaseLoader


class ToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for")


class WikipediaTool(BaseTool):
    name = "WikipediaTool"
    description = """
Use this tool to search keywords in Wikipedia.
It takes a query as an argument.
Example query: Tesla
"""
    args_schema: Type[ToolArgsSchema] = ToolArgsSchema

    def _run(self, query):
        wk = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        return wk.run(query)


class DuckDuckGoTool(BaseTool):
    name = "DuckDuckGoTool"
    description = """
Use this tool to search keywords in DuckDuckGo.
It takes a query as an argument.
Example query: Tesla
"""
    args_schema: Type[ToolArgsSchema] = ToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchResults(api_wrapper=DuckDuckGoSearchAPIWrapper())
        return ddg.run(query)


class ScrapToolArgsSchema(BaseModel):
    weblink: str = Field(description="a link of website")


class ScrapTool(BaseTool):
    name = "ScrapTool"
    description = """
Use this tool to scrap the website.
It takes a weblink as an argument.
"""
    args_schema: Type[ScrapToolArgsSchema] = ScrapToolArgsSchema

    def _run(self, weblink):
        loader = WebBaseLoader(weblink)
        return loader.load()[0].page_content


class FileSaveToolArgsSchema(BaseModel):
    text: str = Field(description="The content to save.")
    file_path: str = Field(description="The file path")


class FileSaveTool(BaseTool):
    name = "FileSaveTool"
    description = """
Use this tool to save input text.
It takes content text and file path as arguments.
"""
    args_schema: Type[FileSaveToolArgsSchema] = FileSaveToolArgsSchema

    def _run(self, text, file_path):
        with open(file_path, "w") as f:
            f.write(text)


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

agent = initialize_agent(
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    tools=[
        WikipediaTool(),
        DuckDuckGoTool(),
        FileSaveTool(),
        ScrapTool(),
    ],
    verbose=True,
    agent_kwargs={
        "system_message": SystemMessage(
            content="""
You are a helpful researcher.
You always save your response to 'search keyword'.txt file.
if you run duckduckgo tool, you should scrap the first web page.
"""
        )
    },
)


agent.invoke("Research about the XZ backdoor")



> Entering new AgentExecutor chain...

Invoking: `DuckDuckGoTool` with `{'query': 'XZ backdoor'}`




c:\Users\kjy70\anaconda3\envs\gpt\Lib\site-packages\langchain\utilities\duckduckgo_search.py:82: UserWarning: backend='api' is deprecated, using backend='auto'
  results = ddgs.text(


[snippet: What Does the Backdoor Do? Malicious code added to xz Utils versions 5.6.0 and 5.6.1 modified the way the software functions. The backdoor manipulated sshd, the executable file used to make remote ..., title: The XZ Backdoor: Everything You Need to Know - WIRED, link: https://www.wired.com/story/xz-backdoor-everything-you-need-to-know/], [snippet: Because the backdoor was discovered before the malicious versions of xz Utils were added to production versions of Linux, "it's not really affecting anyone in the real world," Will Dormann, a ..., title: Backdoor found in widely used Linux utility targets encrypted SSH ..., link: https://arstechnica.com/security/2024/03/backdoor-found-in-widely-used-linux-utility-breaks-encrypted-ssh-connections/], [snippet: CVE-2024-3094 is a vulnerability discovered in the open-source library XZ Utils that stems from malicious code that was pushed into the library by one of its maintainers.. It was originally reported as an SSH authentication bypa